# Multi-Spectral Registration

Copyright (c) 2020, Kostas Alexis, Frank Mascarich, University of Nevada, Reno.

All rights reserved.

This is an IPython Notebook demonstrating usage of the multispectral registration package.

In [ ]:
%matplotlib inline
import time
from multi_spect_tools import multi_spect_common
from multi_spect_tools import multi_spect_image_io
from multi_spect_tools import multi_spect_plotting_utils
from multi_spect_tools import sitk_multi_spect_registration

## Create Registration Object
First, we will create a registration object, passing it a path to a configuration file.

In [ ]:
# Create a registration object
sitk_reg_obj = sitk_multi_spect_registration.sitk_registration("cfg/reg_config.ini")

## Load an image from the dataset
Next, we will load an image from the dataset listed in the configuration file. If the configuration file does not list a dataset, this snippet will fail. Image paths are indexed by their ID, interpretted by their file name. For more information, please see the README section on dataset processing. Note that the paths for dataset processing are not required for image alignment; images may be loaded independently, and simply fed to the alignment function demonstrated in the next code block.

In [ ]:
# Choose an image to load
img_id = 28
# Load the paths to this image
image_paths = sitk_reg_obj.config.get_img_paths(img_id)
# Load the image
input_image = multi_spect_image_io.load_image_from_path_list(image_paths)
# Display the image
multi_spect_plotting_utils.show_multi_ch_image(input_image, sitk_reg_obj.config.ordered_channel_names)

## Run Alignment
Next, we will run the alignment of the loaded image by calling the _align_ function. We pass the input image, and receive the aligned output image, as well as a results object. The results object includes a number of metrics regarding the alignment, as well as the transformation objects applied to retrieve the aligned image. This transformation will be used in subsequent cells as the initial transform for the next image's alignment. 

In [ ]:
output_image, results  = sitk_reg_obj.align(input_image,init_transforms=None, print_output=True)

## Show the Aligned Image
Here we use the plotting utility's function _show_merged_ to display the aligned image. 
The function will plot the image as well as rectangles showing the relative position of each channel.

The function takes:

 1. The aligned image
 2. The indicies of the desired RGB channels
 3. A channel to mix with the RGB
 4. The corner points of each of the aligned channels
 5. The color of each box to draw

In the line below, the red channel is given to exist at the 2nd index of the image, the green at the 1st, and the blue at the 0th. These three channels will be blended (averaged) with the 3rd channel, which in this case is NIR. 

In [ ]:
frame_colors = {'Red':'r', 'Blue':'b', 'Green':'g', 'NIR':'m'}
multi_spect_plotting_utils.show_merged(output_image, [2,1,0], blend_ch=3, image_bounds=results.corner_points, colors=frame_colors)

## Show Convergence Metrics for Each Channel
Below, we plot the convergence values per iteration for each channel. 

In [ ]:
multi_spect_plotting_utils.plot_metric_vals(results.metric_logs, ['b', 'g', 'r'], sitk_reg_obj.config.moving_channel_names, 3)

## Load a 2nd Image
To demonstrate the process of providing an initial transform, we will load and display a 2nd image.

In [ ]:
# Choose an image to load
img_id = 24
# Load the paths to this image
image_paths = sitk_reg_obj.config.get_img_paths(img_id)
# Load the image
input_image = multi_spect_image_io.load_image_from_path_list(image_paths)

## Align the image, using the previous result's transformation
Here we use the *results* object to acquire the previous alignment's final transform and feed it to the alignment of the next image.

In [ ]:
output_image, results  = sitk_reg_obj.align(input_image, init_transforms=results.alignment_transform, print_output=True)

## Show the Aligned Image

In [ ]:
multi_spect_plotting_utils.show_merged(output_image, [2,1,0], 3, results.corner_points, frame_colors)